In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory of src to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db.manager import DBManager
from src.input_to_instructions.load_and_execute import *
from src.input_to_instructions.types import *
from src.plot_graph.execute import *
from src.operation.execute import *
from src.response_generation.load_and_execute import *

INFO:datasets:PyTorch version 2.6.0 available.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from collections import defaultdict, Counter
import logging

import pandas as pd
import numpy as np
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import json
import itertools

from db.manager import DBManager
from operation.execute import OperationExecutor
from pathlib import Path
import warnings
import datetime


warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

INFO:db.instance:Connected to the database PerSite_DB


In [4]:
BASE_DIR = "../"
def read_json(path):
    with open(path, "r", encoding="utf-8") as f:
        result = json.loads(f.read())
    
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in result]
    return result

In [ ]:
ResponseGeneration.initialize(
    log_output=False,
    instance_type="unsloth"
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 2. Max memory: 23.689 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
# def summarize_variables_to_report(variables):
#     for k, v in variables.items():
#         if isinstance(v, pd.DataFrame):
            
#         elif isinstance(v, np.ndarray):
#             variables[k] = v.tolist()
#     return variables


In [7]:
import matplotlib.pyplot as plt
import matplotlib

from src.input_to_instructions.types import InstructionQ_raw, InstructionQ_v2
from src.plot_graph.execute import plot_graph_plotly
matplotlib.rcParams['font.family'] = 'NanumGothicCoding'

def get_time(df, fmt="datetime"):
    # from df get 'timestamp' column and return them in format
    if fmt == "date":
        fmt = '%Y-%m-%d'
    elif fmt == "month":
        fmt = '%Y-%m'
    elif fmt == "year":
        fmt = '%Y'
    else:
        fmt = '%Y-%m-%d %H:%M:%S'
    
    if isinstance(df['timestamp'], pd.Timestamp):
        result = df['timestamp'].strftime(fmt)
    else:
        result = df['timestamp'].apply(lambda x: x.strftime(fmt))
    return sorted(list(set(result)))

def get_spatials(df):
    return pd.unique(df['idu_name'])

def get_tv(df, col:str|list[str], fmt="datetime"):
    if isinstance(col, str):
        col = [col]
    
    timestamps = get_time(df, fmt)
    return_tuple = tuple([timestamps] + [df[c] for c in col])
    return return_tuple

def data(metadata, mapping, query_results, t=str|list[str], s=str|list[str], m=str|list[str]):
    if isinstance(t, str):
        t = [t]
    if isinstance(s, str):
        s = [s]
    if isinstance(m, str):
        m = [m]

    t_raw = [mapping.temporal[t_highlevel] for t_highlevel in t]
    s_raw = [mapping.spatials[s_highlevel] for s_highlevel in s]
    m_raw = [mapping.modalities[m_highlevel] for m_highlevel in m]

    result_df = DBManager.structured_query_data_t_v2(metadata, m_raw, t_raw, s_raw, get_rowids=True)

    cols = list(result_df.columns)
    cols.remove("id")
    cols.remove("idu_name")
    cols.remove("timestamp")
    rows = list(result_df["id"])
    query_results.append({
        "result_columns": cols,
        "result_indices": rows,
    })

    # For demo, drop rows where any value is -1
    result_df = result_df.loc[(result_df != -1).all(axis=1)]

    # drop "id" from result_df
    result_df = result_df.drop(columns=['id'])

    # change column names to high level
    inverse_mapping = {v: k for k, v in mapping.modalities.items()}
    result_df.columns = [inverse_mapping[col] if col in inverse_mapping else col for col in result_df.columns]

    # change idu_name raw values to high level
    inverse_mapping = {v: k for k, v in mapping.spatials.items()}
    result_df["idu_name"] = result_df["idu_name"].map(inverse_mapping)

    return result_df


def run_query_v2(user_input, metadata, instructions):
    instruction_q:InstructionQ_v2 = [i for i in instructions if isinstance(i, InstructionQ_v2)][0]
    mapping = instruction_q
    
    instruction_o:InstructionO = [i for i in instructions if isinstance(i, InstructionO)]
    query_results = []
    if len(instruction_o) != 0:
        instruction_o = instruction_o[0]
        scripts = instruction_o.scripts

        # search data(t=~~, ...,)
        globals()['metadata'] = metadata
        globals()['mapping'] = mapping
        globals()['query_results'] = query_results
        for script in scripts:
            
            if "data" in script:
                script = script.replace("data(", "data(metadata, mapping, query_results, ")
            exec(script, globals())
        
        variables = {name:globals()[name] for name in globals() if name.startswith("v_")}
        instruction_r:InstructionR = [i for i in instructions if isinstance(i, InstructionR)][0]
    else:
        variables = {}
        unknown_spatials = [k for k, v in mapping.spatials.items() if v == "Unknown"]
        unknown_modalities = [k for k, v in mapping.modalities.items() if v == "Unknown"]
        
        response_unknown = f"죄송합니다, {unknown_spatials + unknown_modalities}는 존재하지 않는 공간이나 모달리티 입니다."
        return response_unknown, variables, [], query_results


    response, required_variables = ResponseGeneration.execute(instruction_r, variables, user_input, None, exp_tag=None)
    return response, variables, required_variables, query_results


def run_query(user_input, metadata, instructions, exp_tag=None):
    variables = {
        "Metadata": metadata,
    }
    query_results = []
        
    
    for instruction in instructions:
        # logger.debug(f"Executing instruction: {instruction.__class__.__name__}")
        # print(f"Executing instruction: {instruction.__class__.__name__}")
        
        if type(instruction) == InstructionQ:
            # Execute query
            result_df = DBManager.structured_query_data_t(metadata, instruction.args, get_rowids=True)
            # if result_df is None:
                # print("죄송합니다, 관련 데이터를 찾을 수 없습니다.", "response")
                # return

            cols = list(result_df.columns)
            cols.remove("id")
            cols.remove("idu")
            rows = list(result_df["id"])

            query_results.append({
                "result_columns": cols,
                "result_indices": rows,
            })

            # For demo, drop rows where any value is -1
            result_df = result_df.loc[(result_df != -1).all(axis=1)]

            # drop "id" from result_df
            result_df = result_df.drop(columns=['id'])
           
            #pd.set_option('display.max_rows', 10000)        
            #pd.set_option('display.max_columns', 1000)
            #pd.set_option('display.width', 1000)
            #pd.set_option('display.max_colwidth', 1000)
            #print(f"QueryResult: {result_df}")

            variables[instruction.result_name] = result_df
        elif type(instruction) == InstructionQ_raw:
            instruction.query = instruction.query.replace(" FROM \"data_t\"", ", \"id\" FROM \"data_t\"")
            result_df = DBManager.execute_structured_query_string(
                instruction.query
            )
            # rename idu_name to idu
            result_df = result_df.rename(columns={'idu_name': 'idu'})
            
            cols = list(result_df.columns)
            cols.remove("id")
            cols.remove("idu")
            rows = list(result_df["id"])

            query_results.append({
                "result_columns": cols,
                "result_indices": rows,
            })

            # drop "id" from result_df
            result_df = result_df.drop(columns=['id'])
            
            variables[instruction.result_name] = result_df
            # print(result_df, flush=True)

        elif type(instruction) == InstructionO:
            # Execute operation
            # variables_to_report = {k: v for k, v in variables.items() if k not in ["Metadata"]}
            # print(variables_to_report)
            result_dict = OperationExecutor.execute(variables, instruction.scripts)
            # print(instruction.scripts, instruction.returns, result_dict)
            variables.update(result_dict)
            pass
            # print(fig, "graph")
        elif type(instruction) == InstructionR:
            # Execute response generation
            variables_to_report = {k: v for k, v in variables.items() if k not in ["Metadata"]}
            # print(variables_to_report)
            # variables_to_report = ResponseGeneration.stringify_variables(variables_to_report)
            # variables_to_report = summarize_variables_to_report(variables_to_report)

            # print(f"Variables: {variables_to_report}")

            keys_to_leave = ["modality_mapping", "idu_mapping"]
            metadata_ = {}
            for key in metadata.keys():
                if key in keys_to_leave:
                    metadata_[key] = metadata[key]

            response, required_variables = ResponseGeneration.execute(instruction, variables, user_input, metadata_, exp_tag=exp_tag)
            # print(f"Required variables: {required_variables}")
            
            # response = instruction.expectations[0] # "{{var}}..."
            # for var_name, var_value in required_variables.items():
            #     placeholder = f"{{{{{var_name}}}}}"
            #     if placeholder in response:
            #         response = response.replace(placeholder, str(var_value))

            
            return response, variables_to_report, required_variables, query_results

In [8]:
from copy import deepcopy


def build_query_groundtruth():
    dataset_name = "v7-250309-reduceinputanddatefunctioncall"
    def read(path):
        data = read_json(path)
        for i, d in enumerate(data):
            data[i]["Scenario"] = directory.name
            if "v7" in dataset_name:
                data[i]["Metadata"] = metadata
        return data

    ds_ts = []
    base_dataset_dir = Path(f"{BASE_DIR}/finetuning/dataset/{dataset_name}")
    
    for directory in base_dataset_dir.iterdir():
        if directory.is_dir():
            if "v7" in dataset_name:
                metadata = read_json(f"{directory}/metadata.json")
            
            # d = read(f"{directory}/onlyq_ts.json")
            
            ds_ts.extend(read(f"{directory}/onlyq_ts.json"))
            ds_ts.extend(read(f"{directory}/onlyq_tr.json"))
            # ds_tr.extend(read(f"{directory}/graph.json"))
    
    ds = ds_ts
    print(len(ds))
    # if "v7" in dataset_name:
    #     db_gt_filename = f"{BASE_DIR}/experiments/db_gt_v7.json"
    # else:
    #     db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
    #     metadata = None
    
    # with open(db_gt_filename, "w", encoding="utf-8") as f:
        # f.write("[")
    # with tqdm(total=len(ds)) as pbar:
    
    gts = []

    for d in ds:
        cont = False
        tags = d["Tags"]["Style"]
        skip_tags = ["Reason", "Graph", "Unrelated", "Prediction"]
        for st in skip_tags:
            if st in tags:
                cont = True
                break
        if cont:
            continue

        # pbar.set_description(f"Processing {d['Input']}")
        # print("--")
        exp_tag = "v2"
        # print(f"Warning! exp_tag is v2")
        instructions = InputToInstruction.postprocess(deepcopy(d['Response']), exp_tag=exp_tag)
        user_input, tags, metadata, scenario = d["Input"], d["Tags"], d["Metadata"], d["Scenario"]
        # if user_input != "지금 몇시야?":
        #     continue

        response, variables_to_report, required_variables, query_results = run_query_v2(user_input, metadata, instructions)
        # print(f"출력: {response}")
        # print({k: (v, type(v)) for k, v in variables_to_report.items()})
        gts.append({
            "Input": user_input,
            "Metadata": metadata,
            "Scenario": scenario,
            "Tags": tags,
            "GT": d['Response'],
            "Response": response,
            # "RequiredVariables": required_variables,
            "QueryResults": query_results,
            # "VariablesToReport": variables_to_report,
        })

    # save to json
    with open(f"./gts.json", "w", encoding="utf-8") as f:
        json.dump(gts, f, ensure_ascii=False, indent=4)
  

In [ ]:

build_query_groundtruth()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


38
질문: 이번주 우리반과 앞반의 평균 온도 알려줘; 포맷: ['이번주 우리반의 평균 실내온도({{v_이번주_우리반_실내온도_평균}}℃)와 앞반의 평균 실내온도({{v_이번주_앞반_실내온도_평균}}℃)의 차이는 {{v_이번주_실내온도_평균_차이}}℃입니다.']; 데이터: {'v_이번주_우리반_실내온도_평균': np.float64(25.66922751621088), 'v_이번주_앞반_실내온도_평균': np.float64(25.98149711551991), 'v_이번주_실내온도_평균_차이': np.float64(0.31226959930902964)};
출력: 이번주 우리반의 평균 실내온도(25.67℃)와 앞반의 평균 실내온도(25.98℃)의 차이는 0.31℃입니다.
질문: 현재 설정온도랑 실내온도 차이 알려줘.; 포맷: ['현재 설정온도({{v_현재_우리반_설정온도}}℃)와 실내온도({{v_현재_우리반_실내온도}}℃)의 차이는 {{v_현재_우리반_온도_차이}}℃입니다.']; 데이터: {'v_현재_우리반_설정온도': np.float64(23.0), 'v_현재_우리반_실내온도': np.float64(28.5), 'v_현재_우리반_온도_차이': np.float64(5.5)};
출력: 현재 설정온도(23.00℃)와 실내온도(28.50℃)의 차이는 5.50℃입니다.
질문: 지난달에 설정온도와 실내온도 차이가 가장 많이 났던 날은?; 포맷: ['지난달 {{v_지난달_우리반_온도차이_최고_날짜}}에 설정온도({{v_지난달_우리반_온도차이_최고_설정온도}}℃)와 실내온도({{v_지난달_우리반_온도차이_최고_실내온도}}℃) 차이가 {{v_지난달_우리반_온도차이_최고}}℃로 가장 컸습니다.']; 데이터: {'v_지난달_우리반_온도차이_최고': np.float64(1.0), 'v_지난달_우리반_온도차이_최고_날짜': ['2022-08-19'], 'v_지난달_우리반_온도차이_최고_설정온도': np.float64(23.0), 'v_지난달_우리반_온도차이_최고_실내온도': np.floa

In [12]:
class EM:
    json_structure = "JsonStructureCorrectness"
    true_positive = "QueryTruePositive"
    false_positive = "QueryFalsePositive"
    false_negative = "QueryFalseNegative"
    
def eval_query(cand_response_filename, db_gt_filename="./gts.json"):
    db_gts = read_json(db_gt_filename)
    cand_responses = read_json(cand_response_filename)
    # metadata_ = read_json(f"{BASE_DIR}/finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario1/metadata.json")
    evaluation_reports = []
    response_reports = []
    with tqdm(total=len(cand_responses)) as pbar:
        for cand_response in cand_responses:
            pbar.set_description(f"Processing {cand_response['Input']}")
            input = cand_response["Input"]
            scenario = cand_response["Scenario"]

            # if "오늘 아침과 저녁" not in input:
            #     continue

            if "Metadata" in cand_response:
                metadata = cand_response["Metadata"]
            else:
                # metadata = metadata_
                metadata = None
            # 관계 없는 질문들은 건너뛰자
            gt_report = [d for d in db_gts if d["Input"] == input and d["Scenario"] == scenario]
            assert len(gt_report) <= 1
            if len(gt_report) == 0:
                pbar.update(1)
                continue

            gt_report = gt_report[0]
            tags = gt_report["Tags"]
            # assert gt_report["QueryResults"] != []
            # if gt_report["Result"] == []:
            #     pbar.update(1)
            #     continue
            
            gt_results = [d for d in gt_report["QueryResults"]]
            gt_query_results = defaultdict(list)
            for gt_result in gt_results:
                for col in gt_result["result_columns"]:
                    gt_query_results[col].extend(gt_result["result_indices"])

            gt_total_combinations = sum(len(v) for v in gt_query_results.values())

            gt_response = gt_report["Response"]
            # gt_required_variables = gt_report["RequiredVariables"]
            # gt_variables_to_report = gt_report["VariablesToReport"]
            user_input = gt_report["Input"]

            response_report = {
                "Input": user_input,
                "Metadata": metadata,
                "GT_Response": gt_response,
                # "GT_RequiredVariables": gt_required_variables,
                # "GT_VariablesToReport": gt_variables_to_report,
            }
            # ---
            
            evaluation_report = defaultdict(lambda: None)
            evaluation_report["Input"] = input
            evaluation_report["Metadata"] = metadata
            evaluation_report["Tags"] = tags
            
            if isinstance(cand_response["Candidate"], dict) and ("Instruction Set" in cand_response["Candidate"] or "지시" in cand_response["Candidate"] or "Instructions" in cand_response["Candidate"]):
                if "Instruction Set" in cand_response["Candidate"]:
                    cand_instruction_set = cand_response["Candidate"]["Instruction Set"]
                elif "지시" in cand_response["Candidate"]:
                    cand_instruction_set = cand_response["Candidate"]["지시"]
                elif "Instructions" in cand_response["Candidate"]:
                    cand_instruction_set = cand_response["Candidate"]["Instructions"]

                evaluation_report[EM.json_structure] = True
            else:
                evaluation_report[EM.json_structure] = False
                try:
                    import re
                    # get data between "Instruction Set": [ and the last]
                    cand_instruction_set = re.search(r'(?<="Instructions": \[)(.*)(?=\])', cand_response["Candidate"], re.DOTALL).group(0)
                    # find all {"type": ~ }, {"type": ~ }, {"type": ~ }
                    cand_instruction_set = re.findall(r'({"type".*?})', cand_instruction_set)
                    # print(list(cand_instruction_set))
                    cand_instruction_set = [eval(d) for d in cand_instruction_set]
                except Exception as e:
                    evaluation_report[EM.json_structure] = False
                    evaluation_report[EM.true_positive] = 0
                    evaluation_report[EM.false_positive] = 0
                    evaluation_report[EM.false_negative] = gt_total_combinations

                    print("Failed to parse input: ", input, cand_response["Candidate"])
                    print(e)
                    evaluation_reports.append(evaluation_report)
                    pbar.update(1)
                    response_reports.append(response_report)
                    continue
            
            
            exp_tag = \
                "woCoTExp" if "woCoTExp" in str(cand_response_filename) else \
                "woOp" if "woOp" in str(cand_response_filename) else \
                "woQM" if "woQM" in str(cand_response_filename) else \
                None
            exp_tag = "v2"
            instructions = InputToInstruction.postprocess(
                deepcopy(cand_response["Candidate"]), 
                exp_tag=exp_tag
            )
            
            try:
                # response, variables_to_report, required_variables, _cand_query_results = run_query_v2(user_input, metadata, instructions, exp_tag=exp_tag)
                response, variables_to_report, required_variables, _cand_query_results = run_query_v2(user_input, metadata, instructions)
                print(response)
            except Exception as e:
                print(f"Error: {e}")
                evaluation_report[EM.true_positive] = 0
                evaluation_report[EM.false_positive] = 0
                evaluation_report[EM.false_negative] = gt_total_combinations
                            
                evaluation_reports.append(evaluation_report)

                response_reports.append(response_report)
                            
                pbar.update(1)
                continue
            
            
            # required_variables = summarize_variables_to_report(required_variables)
            required_variables = ResponseGeneration.stringify_variables(required_variables)
            response_report["PD_Response"] = response
            # response_report["PD_RequiredVariables"] = required_variables
            # response_report["PD_VariablesToReport"] = variables_to_report
            response_reports.append(response_report)

            if len(_cand_query_results) == 0:
                evaluation_report[EM.true_positive] = 0
                evaluation_report[EM.false_positive] = 0
                evaluation_report[EM.false_negative] = gt_total_combinations
                            
                evaluation_reports.append(evaluation_report)
                pbar.update(1)
                continue
            
            cand_query_results = defaultdict(list)
            for cand_query_result in _cand_query_results:
                for col in cand_query_result["result_columns"]:
                    cand_query_results[col].extend(cand_query_result["result_indices"])

            cand_total_combinations = sum(len(v) for v in gt_query_results.values())


            if len(gt_results) == 0:
                evaluation_report[EM.true_positive] = 0
                evaluation_report[EM.false_positive] = cand_total_combinations
                evaluation_report[EM.false_negative] = 0

                evaluation_reports.append(evaluation_report)
                pbar.update(1)

                continue
            
            # print(gt_total_combinations, cand_total_combinations)
            # True Positive: 공통된 컬럼과 로우의 모든 조합
            true_positive = 0
            false_negative = 0
            false_positive = 0
            for col in set(gt_query_results.keys())&set(cand_query_results.keys()):
                s_gt_query_result = set(gt_query_results[col])
                s_cand_query_result = set(cand_query_results[col])
                true_positive += len(s_gt_query_result & s_cand_query_result)
                false_negative += len(s_gt_query_result - s_cand_query_result)
                false_positive += len(s_cand_query_result - s_gt_query_result)

                # print(true_positive, false_negative, false_positive, len(s_gt_query_result), len(s_cand_query_result))
            # assert true_positive + false_positive + false_negative == gt_total_combinations
            

            evaluation_report[EM.true_positive] = true_positive
            evaluation_report[EM.false_positive] = false_positive
            evaluation_report[EM.false_negative] = false_negative

            evaluation_reports.append(evaluation_report)
            # print(evaluation_report)
            
            pbar.update(1)

    with open(f"{cand_response_filename.replace('.json', '_response.json')}", "w", encoding="utf-8") as f:
        json.dump(response_reports, f, ensure_ascii=False, indent=4)

    eval_df = pd.DataFrame(evaluation_reports)
    # print(eval_df)

    eval_df['ExactMatch'] = eval_df.apply(lambda x: x[EM.false_positive] == 0 and x[EM.false_negative] == 0, axis=1).astype(int)
    # eval_df['TruePositive'] = eval_df['TruePositive'].astype(int)
    # eval_df['FalsePositive'] = eval_df['FalsePositive'].astype(int)
    # eval_df['FalseNegative'] = eval_df['FalseNegative'].astype(int)

    final_result = {}

    for col in ["JsonStructureCorrectness", "ExactMatch"]:
        # print(f"{col}: {eval_df[col].mean()}")
        final_result[col] = eval_df[col].mean()
    
    # normalize per query
    eval_df["Total"] = eval_df[EM.true_positive] + eval_df[EM.false_positive] + eval_df[EM.false_negative]
    eval_print = eval_df.drop(columns=["Input", "Metadata", "Tags"])
    print(eval_print)
    eval_df[EM.true_positive] = eval_df[EM.true_positive] / eval_df["Total"]
    eval_df[EM.false_positive] = eval_df[EM.false_positive] / eval_df["Total"]
    eval_df[EM.false_negative] = eval_df[EM.false_negative] / eval_df["Total"]

    # # replace nan with 0
    # eval_df.fillna(0, inplace=True)

    # # F1 score except nans.
    truepos_sum, falsepos_sum, falseneg_sum = eval_df[EM.true_positive].sum(), eval_df[EM.false_positive].sum(), eval_df[EM.false_negative].sum()
    precision = truepos_sum / (truepos_sum + falsepos_sum)
    recall = truepos_sum / (truepos_sum + falseneg_sum)
    print(truepos_sum, falsepos_sum, falseneg_sum)
    print(precision, recall)
    f1 = 2 * (precision * recall) / (precision + recall)
    # print(f"F1: {f1}")
    final_result["F1"] = f1
    final_result["Recall"] = recall

    for col in final_result:
        print(f"{col}: {final_result[col]:.2f}")
    
    return eval_df

In [14]:
# name = "r-v7_r256_a512_ours_tr6_0503-checkpoint-63"
# name = "r-v7_r256_a512_ours_tr18_0503-checkpoint-52"
# name = "r-v7_r256_a512_ours_tr30_0503-checkpoint-54"
# name = "r-v7_r256_a512_ours_tr45_0503-checkpoint-95"
# name = "r-v7_r256_a512_ours_tr60_0503-checkpoint-108"

# name = "r-v7_r256_a512_woall_tr6_0503-checkpoint-28"
# name = "r-v7_r256_a512_woall_tr18_0503-checkpoint-70"
# name = "r-v7_r256_a512_woall_tr30_0503-checkpoint-57"
# name = "r-v7_r256_a512_woall_tr45_0503-checkpoint-95"
# name = "r-v7_r256_a512_woall_tr60_0503-checkpoint-90"

names = [
# "r-v7_r256_a512_ours_tr6_0503-checkpoint-63",
# "r-v7_r256_a512_ours_tr18_0503-checkpoint-52",
# "r-v7_r256_a512_ours_tr30_0503-checkpoint-54",
# "r-v7_r256_a512_ours_tr45_0503-checkpoint-95",
# "r-v7_r256_a512_ours_tr60_0503-checkpoint-54",
# "r-v7_r256_a512_woCoT_tr60_0503--checkpoint-84",
# "r-v7_r256_a512_woCoTExp_tr60_0503--checkpoint-102",
# "r-v7_r256_a512_woOp_tr60_0503--checkpoint-90",
# "r-v7_r256_a512_woQM_tr60_0503--checkpoint-54"
# "r-v7_r170_a340_ours_tr56_0613--checkpoint-60",
# "r-v7_r256_a512_ours_tr56_0613--checkpoint-68",
"r-v7_r256_a512_ours_tr17_0613--checkpoint-25"
# "test"
# "v8"
]

for name in names:
    eval_query(
        f"../experiments/{name}.json"
    )

Processing 지금 옆반 온도랑 우리반 온도 알려줘:   0%|          | 0/5 [00:00<?, ?it/s]

질문: 지금 옆반 온도랑 우리반 온도 알려줘; 포맷: ['현재 옆반의 실내온도({{v_현재_옆반_실내온도}}℃)는 우리반의 실내온도({{v_현재_우리반_실내온도}}℃)보다 {{v_현재_옆반_실내온도 - 현재_우리반_실내온도}}℃ 높습니다.']; 데이터: {'v_현재_옆반_실내온도': 0    27.0
Name: 실내온도, dtype: float64, 'v_현재_우리반_실내온도': 0    28.5
Name: 실내온도, dtype: float64};


Processing 1시간 전 설정온도랑 실내온도 차이 알려줘.:  20%|██        | 1/5 [00:01<00:04,  1.19s/it]

현재 옆반의 실내온도(27.00℃)는 우리반의 실내온도(28.50℃)보다 -1.50℃ 낮습니다.
질문: 1시간 전 설정온도랑 실내온도 차이 알려줘.; 포맷: ['1시간 전 설정온도({{v_1시간전_설정온도_실내온도_차이}}℃)는 실내온도({{v_1시간전_실내온도}}℃)보다 {{v_1시간전_설정온도_실내온도_차이}}℃ 높습니다.']; 데이터: {'v_1시간전_설정온도_실내온도_차이': 0     -1.0
1     -2.5
2     -2.5
3     -2.5
4     -2.5
      ... 
175   -2.5
176   -2.5
177   -2.5
178    0.0
179   -2.5
Length: 180, dtype: float64};


Processing 옆반의 가장 최근 온도랑 설정온도 알려줘:  40%|████      | 2/5 [00:02<00:03,  1.09s/it]  

...

27.00℃로, 설정온도는 23.00℃입니다.
질문: 이번주 우리반 평균 온도 알려줘; 포맷: ['이번주 우리반의 평균 온도는 {{v_이번주_우리반_실내온도_평균}}℃ 입니다.']; 데이터: {'v_이번주_우리반_실내온도_평균': np.float64(25.66922751621088)};


Processing 지난 한달간 설정온도 평균을 알려줘.:  80%|████████  | 4/5 [00:03<00:00,  1.24it/s]

이번주 우리반의 평균 온도는 25.67℃ 입니다.
질문: 지난 한달간 설정온도 평균을 알려줘.; 포맷: ['지난 한달간 설정온도 평균은 {{v_지난한달_설정온도_평균}}℃ 입니다.']; 데이터: {'v_지난한달_설정온도_평균': np.float64(22.950033280890718)};


Processing 지난 한달간 설정온도 평균을 알려줘.: 100%|██████████| 5/5 [00:04<00:00,  1.10it/s]

지난 한달간 설정온도 평균은 22.95℃ 입니다.
   JsonStructureCorrectness  QueryTruePositive  QueryFalsePositive  \
0                      True                  2                   0   
1                      True                  0                 360   
2                      True                  2                   0   
3                      True               7094                   0   
4                      True               2865              129343   

   QueryFalseNegative  ExactMatch   Total  
0                   0           1       2  
1                   2           0     362  
2                   0           1       2  
3                   0           1    7094  
4               41136           0  173344  
3.0165278290566735 1.740638835740155 0.24283333520317152
0.6341017756176311 0.9254966470528234
JsonStructureCorrectness: 1.00
ExactMatch: 0.60
F1: 0.75
Recall: 0.93
